<a href="https://colab.research.google.com/github/bforoura/Transformers/blob/main/nlp_transformers_ch6_ex2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Step 1: Activate the GPU

# Go to Runtime / Change runtime type / Hardware Accelerator GPU



In [ ]:
#@title Step 2: Cloning the OpenAI GPT-2 Repository

!git clone https://github.com/openai/gpt-2.git
